<a href="https://colab.research.google.com/github/carmen-herlo/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/CarmenHernandez_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 2 - AG2

Carmen Hernández

https://colab.research.google.com/drive/1VLuAswBfi9nU5vrlmYOpYe-w7mBO-e3Z

https://github.com/carmen-herlo

# Problema viaje por el río. Programación dinámica.

In [0]:
TARIFAS = [
        [0,5,4,3,999,999,999],
        [999,0,999,2,3,999,11],
        [999,999,0,1,999,4,10],
        [999,999,999,0,5,6,9],
        [999,999,999,999,0,999,4],
        [999,999,999,999,999,0,3],
        [999,999,999,999,999,999,0]
        ]

In [0]:
def precios(TARIFAS):
    
    N = len(TARIFAS)
    
    PRECIOS = [[9999]*N for i in [9999]*N]
    RUTAS = [[""]*N for i in [""]*N]
    
    for i in range(N-1):
        for j in range(i+1, N):
            MIN = TARIFAS[i][j]
            RUTAS[i][j] = i #del nodo i al j lo mejor es pasar por i
            
            for k in range(i,j):
                if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
                    MIN = PRECIOS[i][k] + TARIFAS[k][j]
                    RUTAS[i][j] = k
            
            PRECIOS[i][j] = MIN
            
    return PRECIOS, RUTAS

precios, rutas = precios(TARIFAS)

In [0]:
def calcular_ruta(RUTAS, desde, hasta):
    if desde == hasta:
        return desde
    else:
        return str(calcular_ruta(RUTAS, desde, RUTAS[desde][hasta])) + ' - ' + str(RUTAS[desde][hasta])

In [0]:
print(calcular_ruta(rutas, 0, 6))

0 - 0 - 2 - 5


# Problema asignación de tareas. Ramificación y poda.

In [0]:
costes  = [[11,12,18,40], # AGENTE 0
           [14,15,13,22], # AGENTE 1
           [11,17,19,23], # AGENTE 2
           [17,14,20,28]] # AGENTE 3

In [0]:
# retorna el coste de una determinada configuración. Ej. # valor((0,1,2,3), costes)
def valor(s, costes):
    valor = 0
    for i in range(len(s)):
        valor += costes[i][s[i]]
        
    return valor

In [0]:
import itertools

In [0]:
# mejor solución por fuerza bruta
def fuerza_bruta(costes):
    mejor_valor=10e10
    mejor_solucion = ''
    for s in list(itertools.permutations(range(len(costes)))):
        valor_tmp = valor(s, costes)
        if valor_tmp < mejor_valor:
            mejor_solucion = s
            mejor_valor = valor_tmp
    
    print('\tNúmero de permutaciones: ',len(list(itertools.permutations(range(len(costes))))))
    print('\tLa mejor solución es :',mejor_solucion)
    
fuerza_bruta(costes)

	Número de permutaciones:  24
	La mejor solución es : (0, 2, 3, 1)


In [0]:
# =============================================================================
# Ramificación y poda
# =============================================================================

# Coste  inferior para soluciones parciales
# (1,3) se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(s, costes):
    valor = 0
    # valores establecidos
    for i in range (len(s)):
        valor += costes[i][s[i]]
    
    # estimación
    for i in range (len(costes)):
        if i not in s:
            valor += min( [costes[j][i] for j in  range (len(s), len(costes))] )
    
    return valor

print(CI((1,2), costes))

59


In [0]:
#(0,2) ->  (0,2,1),  (0,2,3)
# {'s':(0,2), 'ci':34}
def crear_hijos(nodo, N):
    hijos = []
    for i in range(N):
        if i not in nodo:
            hijos.append({'s':nodo+(i,)})
            
    return hijos

print(crear_hijos((0,2),4))

[{'s': (0, 2, 1)}, {'s': (0, 2, 3)}]


In [0]:
def ramificacion_y_poda(costes):
    dimension= len(costes)
    #(0,1,2,3)
    mejor_solucion = tuple(i for i in range(dimension))
    
    cotaSup = valor(mejor_solucion, costes)
    
    nodos=[]
    nodos.append({'s':(),  'CI': CI((), costes)})
    
    iteracion=0
    
    while(len(nodos) > 0):
        iteracion += 1
        
        nodo_prometedor = min(nodos, key=lambda x:x['CI'])
        
        # hijos
        hijos = [ {'s':x['s'], 'CI':CI(x['s'], costes)} for x in crear_hijos(nodo_prometedor['s'], dimension)]
        
        nodo_final = [ x for x in hijos if len(x['s'])==dimension ]
        if len(nodo_final) > 0:
            if nodo_final[0]['CI']<cotaSup:
                cotaSup = nodo_final[0]['CI']
                mejor_solucion = nodo_final[0]
            
        # poda
        hijos = [ x for x in hijos if x['CI']<cotaSup ]
        
        
        # eliminamos el nodo  expandido
        nodos =  [ x for x in nodos if x['s']!=nodo_prometedor['s'] ]
        
        # añadimos los hijos
        nodos.extend(hijos)
        
    print('\tnº iteraciones: ',iteracion)    
    print('\tLa mejor solución es:',mejor_solucion)
        
    
ramificacion_y_poda(costes)

	nº iteraciones:  93
	La mejor solución es: {'s': (7, 0, 2, 1, 3, 6, 5, 8, 4), 'CI': 15}


# Mejora


Generamos matrices de valores aleatorios de mayores dimensiones y ejecutamos ambos algoritmos.

In [0]:
import numpy as np

valor_min=1 #Valor mínimo de las tareas
valor_max=10 #valor máximo de las tareas
N=[5,6,7,8,9] #lista de dimensiones
for n in N:
    costes=np.random.randint(valor_min, valor_max, size=(n,n))
    print('Matriz de {}x{}'.format(n,n))
    print('Fuerza Bruta:')
    fuerza_bruta(costes)
    print('Ramificación y poda:')
    ramificacion_y_poda(costes)
    print('\n')

Matriz de 5x5
Fuerza Bruta:
	Número de permutaciones:  120
	La mejor solución es : (1, 2, 3, 0, 4)
Ramificación y poda:
	nº iteraciones:  38
	La mejor solución es: {'s': (3, 2, 0, 1, 4), 'CI': 15}


Matriz de 6x6
Fuerza Bruta:
	Número de permutaciones:  720
	La mejor solución es : (2, 1, 4, 5, 3, 0)
Ramificación y poda:
	nº iteraciones:  90
	La mejor solución es: {'s': (3, 5, 4, 1, 2, 0), 'CI': 17}


Matriz de 7x7
Fuerza Bruta:
	Número de permutaciones:  5040
	La mejor solución es : (1, 3, 4, 2, 0, 5, 6)
Ramificación y poda:
	nº iteraciones:  86
	La mejor solución es: {'s': (1, 3, 4, 2, 0, 5, 6), 'CI': 11}


Matriz de 8x8
Fuerza Bruta:
	Número de permutaciones:  40320
	La mejor solución es : (4, 7, 3, 6, 0, 5, 1, 2)
Ramificación y poda:
	nº iteraciones:  177
	La mejor solución es: {'s': (4, 7, 3, 6, 0, 5, 1, 2), 'CI': 22}


Matriz de 9x9
Fuerza Bruta:
	Número de permutaciones:  362880
	La mejor solución es : (7, 0, 8, 3, 2, 4, 1, 6, 5)
Ramificación y poda:
	nº iteraciones:  808
	La mej

Podemos observar cómo al aumentar las dimensiones de la matriz el número de permutaciones crece mucho más rápido que el de iteraciones de ramificación y poda, por lo que, además de ser más lento, su crecimiento exponencial lo hace inviable para matrices de gran tamaño.